In [ ]:
%pip install -r requirements.txt

In [1]:
import os
import numpy as np
import time
import constructivo as constr
from auxiliar_functions import *
from local_search import *

#utilizar xlwt para escribir en excel


In [2]:
directory_path = '../JSSP_Instances'
directory_files = os.listdir(directory_path)
directory_files.sort()

In [3]:
import xlrd, xlwt
from xlutils.copy import copy as xl_copy
from xlwt import Workbook


def write_xlsx(machine_order, makespan, time, name_sheet, name_file):
    # Create a new workbook
    
    try:
        # open existing workbook
        rb = xlrd.open_workbook(name_file, formatting_info=True)
        # make a copy of it
        workbook = xl_copy(rb)
    except:
        workbook = Workbook()
    sheet = workbook.add_sheet(name_sheet)
    
    # Iterate over the matrix and write its values to the sheet
    for row_index, row in enumerate(machine_order, start=1):
        for col_index, value in enumerate(row, start=1):
            sheet.write(row_index - 1, col_index - 1, value)  # Note: row_index and col_index should start from 0
    
    # Write makespan and time to the next row
    row_index = len(machine_order)
    sheet.write(row_index, 0, int(makespan))
    sheet.write(row_index, 1, time)

    # Save the workbook to the specified file
    workbook.save(name_file)

In [4]:
with open('cotas.txt', 'r') as f:
    cotas = f.readlines()
    cotas = [int(line.strip()) for line in cotas]

In [8]:
#Reading all the information
for file_num in range(1, len(directory_files)+1):
  file = "JSSP" + str(file_num) + ".txt"
  with open(directory_path + "/" + file, "r" ) as f:
  #with open("../AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]
  
  # Cota inferior
  machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
  cota_inferior = cotas[file_num-1]

  # Create job objects
  jobs_constr = [constr.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

  #Sorted array in which the first value is always the first machine that 
  machines_constr = [constr.Machine(i+1) for i in range(m)]
  # Record the start time
  constr_start_time = time.time(); ans_constr, machine_order_constr = constr.constructivo(jobs_constr, machines_constr); constr_end_time = time.time()

  rvns_sol = reduced_neighborhood_search(machine_order_constr, processing_time, machines_required, n, m, times_to_test=1000)

  first_swap_start_time = time.time(); first_swap_sol = first_improvement(machine_order_constr, processing_time, machines_required, n, m); first_swap_end_time = time.time()
  best_swap_start_time = time.time(); best_swap_sol = best_improvement(machine_order_constr, processing_time, machines_required, n, m); best_swap_end_time = time.time()

  first_insert_for_start_time = time.time(); first_insert_for_sol = first_improvement(machine_order_constr, processing_time, machines_required, n, m, insert_forward); first_insert_for_end_time = time.time()
  best_insert_for_start_time = time.time(); best_insert_for_sol = best_improvement(machine_order_constr, processing_time, machines_required, n, m, insert_forward); best_insert_for_end_time = time.time()

  first_insert_back_start_time = time.time(); first_insert_back_sol = first_improvement(machine_order_constr, processing_time, machines_required, n, m, insert_backward); first_insert_back_end_time = time.time()
  best_insert_back_start_time = time.time(); best_insert_back_sol = best_improvement(machine_order_constr, processing_time, machines_required, n, m, insert_backward); best_insert_back_end_time = time.time()

  

  print(file, "-> Constr: ", ans_constr, " Cota: ", cota_inferior)
  print( "first swap: ", makespan(first_swap_sol, processing_time, machines_required, n, m), "|",
         "first insert forward: ", makespan(first_insert_for_sol, processing_time, machines_required, n, m), "|" ,
         "first insert backward: ", makespan(first_insert_back_sol, processing_time, machines_required, n, m))
  print( "best swap: ", makespan(best_swap_sol, processing_time, machines_required, n, m), "|",
         "best insert forward: ", makespan(best_insert_for_sol, processing_time, machines_required, n, m), "|" ,
         "best insert backward: ", makespan(best_insert_back_sol, processing_time, machines_required, n, m))
  print(" RVNS -> " , makespan(rvns_sol, processing_time, machines_required, n, m))
  
  #write_xlsx(first_swap_sol, makespan(first_swap_sol, processing_time, machines_required, n, m), first_swap_end_time - first_swap_start_time, file , "JSSP_coberndorm_first_improvement_swap.xlsx")
  #write_xlsx(first_insert_for_sol, makespan(first_insert_for_sol, processing_time, machines_required, n, m), first_insert_for_end_time - first_insert_for_start_time, file, "JSSP_coberndorm_first_improvement_insert_forward.xlsx")
  #write_xlsx(first_insert_back_sol, makespan(first_insert_back_sol, processing_time, machines_required, n, m), first_insert_back_end_time - first_insert_back_start_time, file, "JSSP_coberndorm_first_improvement_insert_backward.xlsx")

  #write_xlsx(best_swap_sol, makespan(best_swap_sol, processing_time, machines_required, n, m), best_swap_end_time - best_swap_start_time, file , "JSSP_coberndorm_best_improvement_swap.xlsx")
  #write_xlsx(best_insert_for_sol, makespan(best_insert_for_sol, processing_time, machines_required, n, m), best_insert_for_end_time - best_insert_for_start_time, file, "JSSP_coberndorm_best_improvement_insert_forward.xlsx")
  #write_xlsx(best_insert_back_sol, makespan(best_insert_back_sol, processing_time, machines_required, n, m), best_insert_back_end_time - best_insert_back_start_time, file, "JSSP_coberndorm_best_improvement_insert_backward.xlsx")

  break


JSSP1.txt -> Constr:  1462  Cota:  991
first swap:  1386 | first insert forward:  1397 | first insert backward:  1386
best swap:  1386 | best insert forward:  1400 | best insert backward:  1386
 RVNS ->  1417


In [39]:
def variable_neighborhood_search(constr, processing_time, machines_required, n, m, methods = [interchange, insert_forward, insert_backward]):
    sol = constr; sol_time = makespan(sol, processing_time, machines_required, n, m)
    stop = False
    sol_best = sol; sol_time_best = sol_time

    while stop == False:
        stop = True
        for mach in range(m):
            for i in range(n):
                for j in range(i+1, n):

                    counter = 0
                    while counter < len(methods):
                        
                        method = methods[counter]
                        new_sol = method(sol, i, j, mach)

                        if feasible(new_sol, machines_required, n, m) == False:
                            continue
                        
                        new_sol_time = makespan(new_sol, processing_time, machines_required, n, m)

                        if new_sol_time < sol_time_best:
                            print(new_sol_time)
                            counter = 0
                            sol_best = new_sol; sol_time_best = new_sol_time
                            stop = False
                        
                        else:
                            print("Entro")
                            counter += 1
                            print(counter)
                    print("Entro 1")
            print("Entro2")
        if stop == False:
            print ("working")
            sol = sol_best; sol_time = sol_time_best
                    
    return sol

In [41]:
variable_neighborhood_search(machine_order_constr, processing_time, machines_required, n, m)

Entro
1
Entro
2
Entro
3
Entro 1
Entro
1
Entro
2
Entro
3
Entro 1


KeyboardInterrupt: 